<a href="https://colab.research.google.com/github/AlesKas/neural_networks/blob/main/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import random

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms

from torchvision.datasets import MNIST
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
BATCH_SIZE = 100
NUM_CLASSES = 10
INPUT_SIZE = 28 * 28
DATASET = MNIST(root='data/', download=True, train=True, transform=transforms.ToTensor())

if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 

In [ ]:
class MnistCNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.out = nn.Linear(32 * 7 * 7, 10)
        self.apply(self._init_weights)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output

    def _init_weights(self, module):
          if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=1.0)
            if module.bias is not None:
                module.bias.data.zero_()


In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.sum(preds == labels).item() / len(preds)

def loss_batch(model, loss_fun, xb, yb, opt=None, metric=None):
    preds = model(xb)

    loss = loss_fun(preds, yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    metric_result = None
    if metric is not None:
        metric_result = metric(preds, yb)

    return loss.item(), len(xb), metric_result

def evaluate(model, loss_fun, valid_dl, metric=None):
    with torch.no_grad():
        results = [loss_batch(model, loss_fun, xb.to(dev), yb.to(dev), metric=metric) for xb, yb in valid_dl]
        losses, nums, metric = zip(*results)

        total = np.sum(nums)

        avg_loss = np.sum(np.multiply(losses, nums)) / total
        avg_metric = None
        if metric is not None:
            avg_metric = np.sum(np.multiply(metric, nums)) / total
        
    return avg_loss, total, avg_metric

def fit(epochs, model, loss_fun, opt, train_dl, valid_dl, metric=None):
    for epoch in range(epochs):
        for xb, yb in train_dl:
            xb, yb = xb.to(dev), yb.to(dev)
            loss, _, _ = loss_batch(model, loss_fun, xb, yb, opt, metric)

        result = evaluate(model, loss_fun, valid_dl, metric)
        val_loss, total, val_metric = result

        if metric is None:
            print(f'Epoch {epoch+1}, loss: {val_loss:.4f}')
        else:
            print(f'Epoch {epoch+1}, loss: {val_loss:.4f}, {metric.__name__}: {val_metric:.4f}')

def split_indices(n, val_pct):
    # Determine size of validation set
    n_val = int(val_pct*n)

    idxs = np.random.permutation(n)

    return idxs[n_val:], idxs[:n_val]

train_indexes, validation_indexes = split_indices(len(DATASET), 0.2)

train_sampler = SubsetRandomSampler(train_indexes)
train_loader = DataLoader(DATASET, BATCH_SIZE, sampler=train_sampler)

val_sampler = SubsetRandomSampler(validation_indexes)
val_loader = DataLoader(DATASET, BATCH_SIZE, sampler=val_sampler)

In [ ]:
model = MnistCNN().to(dev)
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_fun = F.cross_entropy

fit(10, model, loss_fun, optimizer, train_loader, val_loader, accuracy)

Epoch 1, loss: 0.4614, accuracy: 0.8609
Epoch 2, loss: 0.3436, accuracy: 0.8960
Epoch 3, loss: 0.2762, accuracy: 0.9179
Epoch 4, loss: 0.2447, accuracy: 0.9279
Epoch 5, loss: 0.2207, accuracy: 0.9350
Epoch 6, loss: 0.2083, accuracy: 0.9403
Epoch 7, loss: 0.1926, accuracy: 0.9427
Epoch 8, loss: 0.1848, accuracy: 0.9458
Epoch 9, loss: 0.1754, accuracy: 0.9490
Epoch 10, loss: 0.1689, accuracy: 0.9496
Epoch 11, loss: 0.1630, accuracy: 0.9529
Epoch 12, loss: 0.1543, accuracy: 0.9538
Epoch 13, loss: 0.1533, accuracy: 0.9541
Epoch 14, loss: 0.1475, accuracy: 0.9573
Epoch 15, loss: 0.1442, accuracy: 0.9581


In [ ]:
test_dataset = MNIST(root='data/', train=False, transform=transforms.ToTensor())

def predict_image(image, model):
    xb = image.unsqueeze(0).to(dev)
    yb = model(xb)
    yb = yb.to(dev)
    _, preds = torch.max(yb, dim=1)
    return preds[0].item()

for i in range(10):
    img, label = random.choice(test_dataset)
    print(f"Label: {label}, predicted: {predict_image(img, model)}")

test_loader = DataLoader(test_dataset, batch_size=200)
test_loss, total, test_acc = evaluate(model, loss_fun, test_loader, metric=accuracy)
print(f"Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")

Label: 0, predicted: 0
Label: 6, predicted: 6
Label: 3, predicted: 3
Label: 7, predicted: 7
Label: 8, predicted: 8
Label: 0, predicted: 0
Label: 6, predicted: 6
Label: 3, predicted: 3
Label: 1, predicted: 1
Label: 4, predicted: 4
Loss: 0.1293, accuracy: 0.9583
